In [ ]:
from mp_api.client import MPRester
import os

API_KEY = "bSpFQg1jCJpFG4CARe0NiSUyXKke56OF"  # <--- 在这里替换成您的密钥
formula_to_search = "LiAlPO4F"

print(f"开始对 {formula_to_search} 进行分步查询...")

try:
    with MPRester(api_key=API_KEY) as mpr:

        # --- 步骤 1: 查询热力学性质，找到最稳定的材料 ID ---
        # 我们先从热力学端点入手，因为稳定性是首要关心的
        print("\n步骤 1: 查询热力学性质以确定最稳定的结构...")
        thermo_docs = mpr.thermo.search(
            formula=formula_to_search,
            fields=["material_id", "energy_above_hull", "formula_pretty"]
        )

        if not thermo_docs:
            raise ValueError(f"在热力学数据库中未找到关于 {formula_to_search} 的材料。")

        # 按稳定性（energy_above_hull）排序，找到最稳定的那个
        stable_thermo_docs = sorted(thermo_docs, key=lambda doc: doc.energy_above_hull)
        most_stable_thermo_doc = stable_thermo_docs[0]
        
        # 获取最稳定结构的 material_id，这是我们接下来查询的“钥匙”
        stable_material_id = most_stable_thermo_doc.material_id
        energy_above_hull = most_stable_thermo_doc.energy_above_hull
        pretty_formula = most_stable_thermo_doc.formula_pretty

        print(f"找到最稳定结构 ID: {stable_material_id} (稳定性: {energy_above_hull:.3f} eV/atom)")

        # --- 步骤 2: 使用最稳定的 ID 查询电子结构性质（如带隙） ---
        print("\n步骤 2: 使用稳定结构 ID 查询电子结构性质...")
        es_doc = mpr.electronic_structure.search(
            material_ids=[stable_material_id],
            fields=["material_id", "band_gap"]
        )
        
        # es_doc 返回的是列表，我们取第一个
        band_gap = es_doc[0].band_gap if es_doc else None

        # --- 步骤 3: 使用最稳定的 ID 查询其他结构性质 ---
        print("\n步骤 3: 使用稳定结构 ID 查询结构性质...")
        mat_doc = mpr.materials.search(
            material_ids=[stable_material_id],
            fields=["volume", "density", "symmetry"]
        )
        
        # mat_doc 返回的是列表，我们取第一个
        volume = mat_doc[0].volume if mat_doc else None
        density = mat_doc[0].density if mat_doc else None
        crystal_system = mat_doc[0].symmetry.crystal_system.value if mat_doc else None

        # --- 步骤 4: 汇总并打印所有信息 ---
        print("\n--- 查询完成！汇总所有信息 ---")
        print(f"材料 ID (Material ID): {stable_material_id}")
        print(f"化学式 (Formula): {pretty_formula}")
        print(f"晶系 (Crystal System): {crystal_system}")
        print(f"晶胞体积 (Volume): {volume:.2f} Å³" if volume else "N/A")
        print(f"密度 (Density): {density:.2f} g/cm³" if density else "N/A")
        print(f"稳定性 (Energy Above Hull): {energy_above_hull:.3f} eV/atom")
        
        if band_gap is not None:
            if band_gap > 0:
                print(f"带隙 (Band Gap): {band_gap:.3f} eV (这是一个半导体或绝缘体)")
            else:
                print(f"带隙 (Band Gap): {band_gap:.3f} eV (这是一个金属)")
        else:
            print("带隙 (Band Gap): 未提供或查询失败")

except Exception as e:
    print(f"\n查询过程中发生错误: {e}")

In [ ]:
import os
import torch
from mp_api.client import MPRester
import matgl
from matgl.ext.pymatgen import Structure2Graph

# --- 第 0, 1, 2 部分代码保持不变 ---
# ... (省略前面的代码) ...
# --- 第 0 部分: 配置 ---
API_KEY = "bSpFQg1jCJpFG4CARe0NiSUyXKke56OF"  # <--- 在这里替换成您的密钥
MATERIAL_ID = "mp-1210463" 

print("--- 开始执行材料性质预测工作流 ---")

# --- 第一步: 获取晶体结构 ---
print(f"\n步骤 1: 正在从 Materials Project 获取 {MATERIAL_ID} 的晶体结构...")
structure = None
try:
    with MPRester(api_key=API_KEY) as mpr:
        docs = mpr.materials.search(material_ids=[MATERIAL_ID], fields=["structure"])
        if docs:
            structure = docs[0].structure
            print("晶体结构获取成功！")
        else:
            print(f"错误: 无法找到 ID 为 {MATERIAL_ID} 的材料。")
            exit()
except Exception as e:
    print(f"从 Materials Project 获取数据时出错: {e}")
    exit()


# --- 第二步: 加载 MATGL 预训练模型 ---
model_name = "MEGNet-MP-2019.4.1-BandGap-mfi"
print(f"\n步骤 2: 正在加载 MATGL 模型 '{model_name}'...")
model = matgl.load_model(model_name)
print("模型加载成功！")


# --- 第三步: 将结构转换为图，并进行必要的修正 ---
if structure:
    print(f"\n步骤 3: 正在将晶体结构转换为图并修正数据...")
    
    converter = Structure2Graph(element_types=model.model.element_types, cutoff=5.0)
    
    # get_graph 返回一个元组，我们只需要第一个元素，即图对象 g
    g = converter.get_graph(structure)[0] 
    
    # 修正数据类型
    if 'node_type' in g.ndata and g.ndata['node_type'].dtype != torch.long:
        g.ndata['node_type'] = g.ndata['node_type'].to(torch.long)

    # 修正节点数据键名
    if 'frac_coords' in g.ndata:
        g.ndata['pos'] = g.ndata.pop('frac_coords')

    # 修正边数据键名
    if 'pbc_offset' in g.edata:
        g.edata['pbc_offshift'] = g.edata.pop('pbc_offset')
    
    print("数据准备完成！")

    # --- 第四步: 使用图对象进行预测 ---
    print(f"\n步骤 4: 正在对 {structure.formula} ({MATERIAL_ID}) 进行带隙预测...")
    
    # ####################################################################
    #
    #   最 终 关 键 修 正
    #   调用模型时，不再传递 state_attr 参数
    #
    # ####################################################################
    predicted_band_gap = model(g)
    
    band_gap_value = predicted_band_gap.item()
    
    print("\n--- 预测完成 ---")
    print(f"MATGL 模型预测的带隙为: {band_gap_value:.3f} eV")

In [ ]:
import torch
import matgl
from pymatgen.core import Lattice, Structure

# --- 步骤 1: 创建一个晶体结构作为输入 ---
# 这里我们使用官方文档中的 CsCl 结构作为示例。
# 您的任何其他 pymatgen 结构对象都可以替换这里的 'struct'。
print("正在创建 CsCl 晶体结构...")
struct = Structure.from_spacegroup(
    "Pm-3m", Lattice.cubic(4.1437), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]]
)
print("结构创建成功。")

# --- 步骤 2: 加载预训练的带隙预测模型 ---
# 我们加载文档中指定的、需要额外条件（state_attr）的多保真度模型。
model_name = "MEGNet-MP-2019.4.1-BandGap-mfi"
print(f"正在加载模型 '{model_name}'...")
model = matgl.load_model(model_name)
print("模型加载成功。")

# --- 步骤 3: 进行预测 ---
# 对于这个特殊的多保真度模型，我们需要提供一个 state_attr 来选择预测哪种计算精度的带隙。
# 0 代表 PBE 精度，这是最常用的一种。
state_attr_for_pbe = torch.tensor([0])
print("正在使用 PBE 精度 (state_attr = 0) 进行带隙预测...")

# 直接调用高级预测接口 predict_structure
predicted_band_gap = model.predict_structure(
    structure=struct, 
    state_attr=state_attr_for_pbe
)

# --- 最终结果 ---
# 使用 .item() 从结果张量中提取出纯数字
band_gap_value = predicted_band_gap.item()

print("\n--- 预测完成 ---")
print(f"模型预测的 CsCl (PBE) 带隙为: {band_gap_value:.3f} eV")

### 提取材料学中几个open-ended类型的数据各50条进行标注

In [ ]:
import json
from collections import defaultdict
import os
raw_data_path= os.path.join("raw_data")
data_path= os.path.join(raw_data_path, "SciKnowEval_processed_tag.json")

num_dict = defaultdict(int)
# 读取 JSON 文件
data_path= os.path.join(raw_data_path, "SciKnowEval_processed.json")
with open(data_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
output_list = []
# 遍历数据
for item in data:
    if item["metadata"]["domain"]=="Material":
        subtask = item["metadata"]["details"]["subtask"]
        if subtask=="material_component_extraction" \
        or subtask==   "property_and_usage_analysis":
        
        # or subtask== "crystal_structure_and_composition_analysis"\
            # 统计子任务出现的次数
            if num_dict[subtask]<50:
                num_dict[subtask] += 1
                # 将 item 添加到输出列表
                item["generations"][0]["evaluation"]["tag"]=item["generations"][0]["evaluation"]["correctness"]
                output_list.append(item)
# 将输出列表写入新的 JSON 文件
output_path = os.path.join(raw_data_path, "SciKnowEval_processed_tag.json")
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(output_list, file, ensure_ascii=False, indent=4)
            
       
        

In [ ]:
import os
from mp_api.client import MPRester

# --- 配置您的 API 密钥 ---
# 强烈建议将密钥设置为环境变量 MP_API_KEY
# 如果已经设置，MPRester() 会自动读取，无需传入参数
API_KEY = "bSpFQg1jCJpFG4CARe0NiSUyXKke56OF"  # <--- 请在这里替换成您自己的密钥

# 目标化学式
formula_to_search = "LiFePO4F"

print(f"--- 步骤 1: 获取 {formula_to_search} 的晶体结构 ---")
print("正在连接 Materials Project 数据库...")

# 将最终的 pymatgen Structure 对象保存在这里
LiFePO4F_structure = None

try:
    with MPRester(api_key=API_KEY) as mpr:
        # 首先，从热力学端点查询，找到最稳定的材料ID
        print(f"正在查询 {formula_to_search} 的热力学稳定性...")
        thermo_docs = mpr.thermo.search(
            formula=formula_to_search,
            fields=["material_id", "energy_above_hull", "formula_pretty"]
        )

        if not thermo_docs:
            raise ValueError(f"在热力学数据库中未找到关于 {formula_to_search} 的材料。")

        # 按稳定性排序，找到能量最低的（最稳定的）条目
        stable_thermo_docs = sorted(thermo_docs, key=lambda doc: doc.energy_above_hull)
        most_stable_entry = stable_thermo_docs[0]
        
        stable_material_id = most_stable_entry.material_id
        stability = most_stable_entry.energy_above_hull

        print(f"已找到最稳定的结构: {stable_material_id} (稳定性 E-above-hull: {stability:.3f} eV/atom)")

        # 然后，使用最稳定的ID去获取其详细的晶体结构
        print(f"正在使用 ID ({stable_material_id}) 获取详细结构...")
        structure_docs = mpr.materials.search(
            material_ids=[stable_material_id], 
            fields=["structure"]
        )
        
        if structure_docs:
            LiFePO4F_structure = structure_docs[0].structure
            print("晶体结构获取成功！")
        else:
            raise ValueError(f"无法使用ID {stable_material_id} 获取结构信息。")

except Exception as e:
    print(f"\n查询过程中发生错误: {e}")

# --- 打印最终获取的结构信息 ---
if LiFePO4F_structure:
    print("\n--- LiFePO4F (最稳定结构) 详细信息 ---")
    print(LiFePO4F_structure)

In [ ]:
import os
import torch
from mp_api.client import MPRester
import matgl
from matgl.ext.pymatgen import Structure2Graph
from  matgl.ext.ase import Relaxer # 导入专门用于结构弛豫的工具

# ==============================================================================
# 步骤 1: 一次性加载所有需要的模型
# ==============================================================================
print("--- 正在加载所有必需的 MATGL 模型 ---")

pes_model = matgl.load_model("CHGNet-MPtrj-2024.2.13-11M-PES")
print("势能面 (PES) 模型加载成功！")

bandgap_model_wrapper = matgl.load_model("MEGNet-MP-2019.4.1-BandGap-mfi")
bandgap_model = bandgap_model_wrapper.model 
element_types = bandgap_model.element_types
print("带隙 (BandGap) 模型加载成功！")


# ==============================================================================
# 步骤 2: 定义一个可重复使用的、稳健的预测函数
# ==============================================================================
def predict_property_for_modified_structure(
    initial_structure,
    modification_type: str,
    modification_details: dict,
    potential_model, 
    property_model,
):
    """
    一个通用的函数，用于修改结构、进行弛豫，并预测最终性质。
    """
    print(f"\n--- 开始处理新任务：对 {initial_structure.composition.reduced_formula} 进行 {modification_type} 操作 ---")
    
    # --- 2.1. 根据指令修改结构 ---
    print("步骤 2.1: 正在应用原子修改...")
    modified_structure = initial_structure.copy()
    
    if modification_type == "substitute":
        from_el = modification_details["from"]
        to_el = modification_details["to"]
        modified_structure.replace_species({from_el: to_el})
        print(f"已将 {from_el} 替换为 {to_el}。新化学式: {modified_structure.composition.reduced_formula}")
    # ... (可以补充 'remove', 'add' 等其他修改类型的逻辑)
    else:
        raise ValueError(f"不支持的修改类型: {modification_type}")

    # --- 2.2. 对新结构进行弛豫，找到稳定构型 ---
    print("步骤 2.2: 正在使用 PES 模型进行结构弛豫...")
    relaxer = Relaxer(potential=potential_model)
    relaxation_results = relaxer.relax(modified_structure, fmax=0.1)
    
    # ####################################################################
    #
    #   根 据 官 方 文 档 修 正 在 这 里
    #   从轨迹的能量列表 .energies 中获取最后一个值
    #
    # ####################################################################
    final_energy = relaxation_results["trajectory"].energies[-1]
    
    relaxed_structure = relaxation_results["final_structure"]
    print(f"结构弛豫完成。最终能量: {float(final_energy):.3f} eV")

    # --- 2.3. 使用弛豫后的结构预测最终性质 ---
    # (这部分代码已在之前调试正确，保持不变)
    print("步骤 2.3: 正在对弛豫后的结构预测带隙...")
    converter = Structure2Graph(element_types=element_types, cutoff=5.0)
    graph_tuple = converter.get_graph(relaxed_structure)
    g = graph_tuple[0]
    state_attr = torch.tensor([0]) # 使用 PBE 精度进行预测
    
    # 修正数据类型和键名...
    if 'node_type' in g.ndata and g.ndata['node_type'].dtype != torch.long:
        g.ndata['node_type'] = g.ndata['node_type'].to(torch.long)
    if 'frac_coords' in g.ndata:
        g.ndata['pos'] = g.ndata.pop('frac_coords')
    if 'pbc_offset' in g.edata:
        g.edata['pbc_offshift'] = g.edata.pop('pbc_offset')
    
    # 调用核心模型进行预测
    final_prediction = property_model(g, state_attr=state_attr)
    
    return final_prediction.item()

if __name__ == "__main__":
    # --- 3.1 获取一个初始结构作为示例 ---
    API_KEY = "bSpFQg1jCJpFG4CARe0NiSUyXKke56OF" # 已使用您提供的密钥
    initial_structure = None
    formula= "LiFePO4F"  # 目标化学式
    with MPRester(api_key=API_KEY) as mpr:
        # 以 LiFePO4F (mp-755813) 为例
        docs = mpr.materials.search(formula=formula, fields=["structure"])
        if docs:
            initial_structure = docs[0].structure
    parsed_json = {
        "modification_type": "substitute",
        "new_material_formula": "LiAlPO4F", # 我们可以在最后打印时使用
        "details": {
            "from_element": "Fe",
            "to_element": "Al"
        }
    }
    if initial_structure:
        modification_details_for_function = {
            "from": parsed_json["details"].get("from_element"),
            "to": parsed_json["details"].get("to_element"),
            "element": parsed_json["details"].get("element"),
            "coords": parsed_json["details"].get("coords")
        }

        # 这就是您在并发任务中需要调用的核心部分
        predicted_bandgap = predict_property_for_modified_structure(
            initial_structure=initial_structure,
            modification_type=parsed_json["modification_type"], # 使用 'modification_type' 键
            modification_details=modification_details_for_function, # 使用转换后的details字典
            potential_model=pes_model,
            property_model=bandgap_model
        )
        
        # 在打印最终结果时，我们可以用上JSON里的 'new_material_formula'
        final_formula = parsed_json.get("new_material_formula", "N/A")

        print("\n==============================================")
        print(f"最终预测结果：{final_formula} 的带隙约为 {predicted_bandgap:.3f} eV")
        print("==============================================")

### 抽取一批数据，测试MEGNet模型的准确性

In [1]:
import os
import torch
import random
import numpy as np
from mp_api.client import MPRester
import matgl

print("--- 步骤 1: 全局配置和模型加载 ---")
API_KEY = "bSpFQg1jCJpFG4CARe0NiSUyXKke56OF" 


# 直接加载完整的模型包装器，我们将使用它的高级接口
bandgap_model_wrapper = matgl.load_model("MEGNet-MP-2019.4.1-BandGap-mfi")
print("带隙 (BandGap) 模型加载成功！")



with MPRester(api_key=API_KEY) as mpr:
    # --- 3.1 从数据库中进行广泛搜索 ---

    
    fields_to_request = ["material_id", "formula_pretty", "band_gap"]
    
    # 使用正确的关键字参数和元组进行范围查询
    candidate_docs = mpr.materials.summary.search(
        energy_above_hull=(None, 0.05),
        band_gap=(0.1, None),
        fields=fields_to_request
    )





/u01/mengpengyu/miniconda3/envs/d2l/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 步骤 1: 全局配置和模型加载 ---


/u01/mengpengyu/miniconda3/envs/d2l/lib/python3.10/site-packages/matgl/utils/io.py:148: UserWarning: Incompatible model version detected! The code will continue to load the model but it is recommended that you provide a path to an updated model, increment your @model_version in model.json if you are confident that the changes are not problematic, or clear your ~/.matgl cache using `python -c "import matgl; matgl.clear_cache()"`
  _check_ver(cls_, v)  # Check version of any subclasses too.


带隙 (BandGap) 模型加载成功！


Retrieving SummaryDoc documents: 100%|██████████| 42390/42390 [00:26<00:00, 1575.13it/s]


In [2]:
SAMPLE_SIZE = 50
random_sample = random.sample(candidate_docs, SAMPLE_SIZE)
structure_list=[]
for i, entry in enumerate(random_sample):
    material_id = entry.material_id

    structure_doc = mpr.materials.search(material_ids=[material_id], fields=["structure"])[0]
    structure = structure_doc.structure
    structure_list.append(structure)

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 5146.39it/s]


In [ ]:
def get_model_prediction(structure, model_wrapper):
    """
    一个简化的函数，直接使用模型的高级接口 predict_structure 进行预测。
    
    Args:
        structure: pymatgen Structure 对象。
        model_wrapper: 加载的完整 matgl 模型（包含转换器）。
    """
    # 对于这个多保真度模型，我们使用 state_attr=0 来预测 PBE 带隙，
    # 以便和 Materials Project 数据库中的默认带隙值进行对比。
    state_attr = torch.tensor([3])
    
    # 直接调用高级接口，所有的数据转换和修正都在后台自动完成
    prediction_tensor = model_wrapper.predict_structure(
        structure=structure,
        state_attr=state_attr
    )
    
    return prediction_tensor.item()

all_results=[]


print(f"已成功筛选出 {len(random_sample)} 个材料用于测试。")

# --- 3.2 遍历样本，进行预测和对比 ---
print("\n--- 步骤 3.2: 开始逐个预测并对比结果 ---")
for i, entry in enumerate(random_sample):
    material_id = entry.material_id
    formula = entry.formula_pretty
    true_band_gap = entry.band_gap

    print(f"\n({i+1}/{SAMPLE_SIZE}) 正在处理: {formula} ({material_id})")
    print(f"  数据库真实带隙 (DFT): {true_band_gap:.3f} eV")

    try:
        structure = structure_list[i]
        
        # 调用我们简化的预测函数
        predicted_band_gap = get_model_prediction(structure, bandgap_model_wrapper)
        print(f"  模型预测带隙 (ML):   {predicted_band_gap:.3f} eV")
        
        # 保存结果
        all_results.append({
            "id": material_id,
            "formula": formula,
            "true_gap": true_band_gap,
            "predicted_gap": predicted_band_gap
        })
    except Exception as e:
        print(f"  处理 {material_id} 时出错: {e}。跳过此样本。")

# --- 3.3 计算并输出最终的准确性指标 ---
print("\n--- 步骤 3.3: 计算模型整体准确性 ---")
if all_results:
    true_values = np.array([res["true_gap"] for res in all_results])
    predicted_values = np.array([res["predicted_gap"] for res in all_results])
    
    mae = np.mean(np.abs(true_values - predicted_values))
    
    print("\n================ 最终评估结果 ================")
    print(f"成功测试了 {len(all_results)} 个样本。")
    print(f"平均绝对误差 (MAE): {mae:.3f} eV")
    print("==============================================")
    print("\n*注: MAE 是“预测值”与“真实值”之间差值的平均数，这个值越小，说明模型预测越准确。")
else:
    print("没有成功完成的测试样本，无法计算准确性。")

In [5]:
import os
import torch
import json
from mp_api.client import MPRester
from alignn.models.alignn import ALIGNN, ALIGNNConfig
from alignn.models.alignn_atomwise import ALIGNNAtomWise, ALIGNNAtomWiseConfig # <--- 导入正确的模型类
from alignn.graphs import Graph
from pymatgen.core import Structure


structure= structure_list[0]  # 假设我们使用第一个结构进行预测
true_band_gap = random_sample[0].band_gap  # 获取数据库中的真实带隙值
model_name_to_use = 'mp_gappbe_alignn' # 这是我们从列表中找到的正确模型名
# model_name_to_use = 'jv_mbj_bandgap_alignn'  # 这是我们从列表中找到的正确模型名
        # 加载模型的配置文件 (config.json)
if structure:
    model_name_to_use = 'mp_gappbe_alignn'
    MODEL_PATH = os.path.expanduser(f"~/.cache/alignn/{model_name_to_use}")

    print(f"\n--- 步骤2: 正在从本地路径 '{MODEL_PATH}' 加载 ALIGNN 模型... ---")
    
    try:
        # 加载模型的配置文件 (config.json)
        config_path = os.path.join(MODEL_PATH, "config.json")
        with open(config_path, "r") as f:
            config_json = json.load(f)

        # ####################################################################
        #
        #   根 据 ValidationEror 的 最 终 修 正
        #   在创建配置对象前，手动修正 'name' 字段以满足 ALIGNNAtomWiseConfig 的要求
        #
        # ####################################################################
        config_json['model']['name'] = 'alignn_atomwise'
        
        # 使用修正后的 'model' 子字典来创建配置对象
        config = ALIGNNAtomWiseConfig(**config_json['model'])
        
        # 使用 ALIGNNAtomWise 来构建模型框架
        model = ALIGNNAtomWise(config)
        
        # 加载预训练好的模型权重 (model.pth)
        state = torch.load(os.path.join(MODEL_PATH, "checkpoint_300.pt"), map_location=torch.device("cpu"))
        model.load_state_dict(state["model"],strict=False)
        
        print("模型从本地加载成功！")
        
        # --- 4. 将结构转换为图并进行预测 ---
        print("\n--- 步骤3: 正在准备图数据并进行预测... ---")
        
        # 将 pymatgen Structure 转换为 ASE Atoms 对象，再转为 dgl 图
        atoms = structure.to_ase_atoms()
        g, lg = Graph.atom_dgl_multigraph(atoms)
        
        model.eval() 
        with torch.no_grad():
            # 直接调用模型，传入图对象
            prediction = model([g, lg]) 
            predicted_band_gap = prediction['out'].item()

        print("\n--- 预测完成 ---")
        print(f"ALIGNN 模型预测的带隙为: {predicted_band_gap:.3f} eV")
    
        # --- 5. 对比结果 ---
        print("\n--- 结果对比 ---")
        error = abs(predicted_band_gap - true_band_gap)
        print(f"模型预测值与数据库真实值的绝对误差为: {error:.3f} eV")

    except Exception as e:
        import traceback
        print(f"\n加载或预测过程中发生错误:")
        traceback.print_exc()

        
  


--- 步骤2: 正在从本地路径 '/u01/mengpengyu/.cache/alignn/mp_gappbe_alignn' 加载 ALIGNN 模型... ---
模型从本地加载成功！

--- 步骤3: 正在准备图数据并进行预测... ---

加载或预测过程中发生错误:


Traceback (most recent call last):
  File "/tmp/ipykernel_1422611/4223934370.py", line 53, in <module>
    g, lg = Graph.atom_dgl_multigraph(atoms)
  File "/u01/mengpengyu/miniconda3/envs/d2l/lib/python3.10/site-packages/alignn/graphs.py", line 491, in atom_dgl_multigraph
    edges, images = nearest_neighbor_edges(
  File "/u01/mengpengyu/miniconda3/envs/d2l/lib/python3.10/site-packages/alignn/graphs.py", line 164, in nearest_neighbor_edges
    all_neighbors = atoms.get_all_neighbors(r=cutoff)
AttributeError: 'MSONAtoms' object has no attribute 'get_all_neighbors'


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model_id = "ysuz/Mistral-Nemo-Base-2407-bandgap"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                            )

# example of input context
structure_text = """
Reduced Formula: BaSrI4
abc   :   5.807091   5.807091   8.251028
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
space group: ('P4/mmm', 123)
Sites (6)
  #  SP      a    b         c    magmom
  0  Ba    0.5  0.5  0               -0
  1  Sr    0    0    0.5             -0
  2  I     0    0.5  0.257945         0
  3  I     0.5  0    0.257945         0
  4  I     0    0.5  0.742055         0
  5  I     0.5  0    0.742055         0

Output:
"""

prompt = f"Instruction: What is the bandgap value of following material?:\n{structure_text}\n\nOutput:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        repetition_penalty=1.05,
    )
generated_text = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(f"Generated raw text:\n{generated_text}\n\n")


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like ysuz/Mistral-Nemo-Base-2407-bandgap is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.